In [1]:
import random
from random import shuffle

In [2]:
random.randint(0,1)

1

In [4]:
# Number of items

# Number of training items for each n
# L&P did 144: 48 for n=1, 48 for n=2, 48 for n=3
count_train_1 = 16 
count_train_2 = 16 
count_train_3 = 16

count_test_seen = 10
count_test_good = 10
count_test_nm = 5
count_test_nodep = 5
count_test_long = 5
count_test_longone = 5
count_test_long_nodep = 5
count_test_atob = 5


# Whether to allow sequences to repeat within the training set
# Note: Have to allow repeats for n = 1
sequence_repeats = False

# Whether to allow syllables to repeat within a particular training item
syll_repeats = False

# Whether we are starting small - i.e., whether the training set should be arranged by size
starting_small = False

# Whether the buttons are arranged in order or not
buttons_ordered = True

In [5]:
def count_in_list(lst, item):
    count = 0
    for elt in lst:
        if elt == item:
            count += 1
            
    return count

In [6]:
syll_dict = {}
syll_dict["b"] = ["be", "bi"]
syll_dict["d"] = ["de", "di"]
syll_dict["g"] = ["ge", "gi"]
syll_dict["p"] = ["po", "pu"]
syll_dict["t"] = ["to", "tu"]
syll_dict["k"] = ["ko", "ku"]

ones = [["b", "p"], ["d", "t"], ["g", "k"]]

def gen_good(half_length):
    full_onsets = []
    
    for i in range(half_length):
        if syll_repeats:
            new_pair = random.choice(ones)
            full_onsets = [new_pair[0]] + full_onsets + [new_pair[1]]
        else:
            satisfied = False
            while not satisfied:
                candidate_pair = random.choice(ones)
                if count_in_list(full_onsets, candidate_pair[0]) < 2:
                    satisfied = True
                    full_onsets = [candidate_pair[0]] + full_onsets + [candidate_pair[1]]
                
    words = []
    for onset in full_onsets:
        if syll_repeats:
            words.append(random.choice(syll_dict[onset]))
        else:
            satisfied = False
            while not satisfied:
                candidate_syll = random.choice(syll_dict[onset])
                if candidate_syll not in words:
                    words.append(candidate_syll)
                    satisfied = True
            
    return words
    

In [7]:
def generate_training_set(count_1, count_2, count_3, count_4, list_to_avoid, no_repeats_period=False):
    training_set = []
    training_dict = {}
    for elt in list_to_avoid:
        training_dict[tuple(elt)] = 1
    
    for i in range(count_1):
        train_item = gen_good(1)
        
        if not sequence_repeats:
            satisfied = False
            while not satisfied:
                train_item = gen_good(1)
                if tuple(train_item) not in training_dict:
                    satisfied = True
        
        training_set.append(train_item)
        
        if no_repeats_period:
            training_dict[tuple(train_item)] = 1
        
    for i in range(count_2):
        train_item = gen_good(2)
        
        if not sequence_repeats:
            satisfied = False
            while not satisfied:
                train_item = gen_good(2)
                if tuple(train_item) not in training_dict:
                    satisfied = True
        
        training_set.append(train_item)
        training_dict[tuple(train_item)] = 1
        
    for i in range(count_3):
        train_item = gen_good(3)
        
        if not sequence_repeats:
            satisfied = False
            while not satisfied:
                train_item = gen_good(3)
                if tuple(train_item) not in training_dict:
                    satisfied = True
        
        training_set.append(train_item)
        training_dict[tuple(train_item)] = 1
        
    for i in range(count_4):
        train_item = gen_good(4)
        
        if not sequence_repeats:
            satisfied = False
            while not satisfied:
                train_item = gen_good(4)
                if tuple(train_item) not in training_dict:
                    satisfied = True
        
        training_set.append(train_item)
        training_dict[tuple(train_item)] = 1
        
    if not starting_small:
        shuffle(training_set)
        
    return training_set

In [8]:
def triple_summing_to(overall_count):
    count_1 = random.randint(0,overall_count)
    count_2 = random.randint(0,overall_count - count_1)
    count_3 = overall_count - count_1 - count_2
    
    return count_1, count_2, count_3

In [9]:
def generate_test_good(overall_count):
    count_1, count_2, count_3 = triple_summing_to(overall_count)
    
    return generate_training_set(count_1, count_2, count_3, 0, [])

In [10]:
starters = ["be", "bi", "de", "di", "ge", "gi"]
enders = ["po", "pu", "to", "tu", "ko", "ku"]

def generate_nm_short(overall_count):
    count_2 = random.randint(0,overall_count)
    count_3 = overall_count - count_2
    
    to_modify = generate_training_set(0, count_2, count_3, 0, [])
    
    to_return = []
    
    for elt in to_modify:
        starting = random.choice([True, False])
        if starting:
            to_return.append(elt[:-1])
        else:
            to_return.append(elt[1:])
            
    return to_return

def generate_nm_long(overall_count):
    count_4 = overall_count
    
    to_modify = generate_training_set(0, 0, 0, count_4, [])
    
    to_return = []
    
    for elt in to_modify:
        starting = random.choice([True, False])
        if starting:
            to_return.append(elt[:-1])
        else:
            to_return.append(elt[1:])
            
    return to_return

In [11]:
def generate_nodep_short(overall_count):
    count_1, count_2, count_3 = triple_summing_to(overall_count)
    
    to_modify = generate_training_set(count_1, count_2, count_3, 0, [])
    
    to_return = []
    
    for elt in to_modify:
        if len(elt) == 2 or len(elt) == 4:
            position = 0
        elif len(elt) == 6:
            position = random.choice([0,1])
        else:
            print("BAD LENGTH")
            
        to_replace = elt[position]
        
        if to_replace[0] == "b":
            new_start = random.choice(["d", "g"])
        elif to_replace[0] == "d":
            new_start = random.choice(["b", "g"])
        elif to_replace[0] == "g":
            new_start = random.choice(["b", "d"])
        else:
            print("BAD ONSET")
            
        new_syll = random.choice(syll_dict[new_start])
        elt[position] = new_syll
        
        to_return.append(elt)
            
            
    return to_return

def generate_nodep_long(overall_count):
    count_4 = overall_count
    
    to_modify = generate_training_set(0, 0, 0, count_4, [])
    
    to_return = []
    
    for elt in to_modify:
        position = random.choice([0,1,2])
            
        to_replace = elt[position]
        
        if to_replace[0] == "b":
            new_start = random.choice(["d", "g"])
        elif to_replace[0] == "d":
            new_start = random.choice(["b", "g"])
        elif to_replace[0] == "g":
            new_start = random.choice(["b", "d"])
        else:
            print("BAD ONSET")
            
        new_syll = random.choice(syll_dict[new_start])
        elt[position] = new_syll
        
        to_return.append(elt)
            
            
    return to_return


In [12]:
def generate_unordered_short(overall_count):
    count_1, count_2, count_3 = triple_summing_to(overall_count)
    
    to_modify = generate_training_set(count_1, count_2, count_3, 0, [])
    
    to_return = []
    
    end_change = random.choice([True, False])
    
    for elt in to_modify:
        if len(elt) == 2 or len(elt) == 4:
            position = 0
        elif len(elt) == 6:
            position = random.choice([0,1])
        else:
            print("BAD LENGTH")
            
        if end_change:
            elt = elt[::-1]
            
        to_replace = elt[position]
        
        if to_replace[0] == "b":
            new_start = "p"
        elif to_replace[0] == "d":
            new_start = "t"
        elif to_replace[0] == "g":
            new_start = "k"
        elif to_replace[0] == "p":
            new_start = "b"
        elif to_replace[0] == "t":
            new_start = "d"
        elif to_replace[0] == "k":
            new_start = "g"
        else:
            print("BAD ONSET")
            
        new_syll = random.choice(syll_dict[new_start])
        elt[position] = new_syll
        
        if end_change:
            elt = elt[::-1]
        
        to_return.append(elt)
            
            
    return to_return


In [13]:
generate_unordered_short(5)

[['de', 'ge', 'ko', 'di'],
 ['be', 'be'],
 ['de', 'de'],
 ['bi', 'di', 'de', 'tu', 'di', 'po'],
 ['de', 'bi', 'ge', 'ko', 'bi', 'tu']]

In [14]:
def choose_uniqe_subset(lst, count):
    chosen_set = {}
    chosen_lst = []
    
    for i in range(count):
        satisfied = False
        while not satisfied:
            chosen = random.choice(lst)
            if tuple(chosen) not in chosen_set:
                satisfied = True
        chosen_lst.append(chosen)
        chosen_set[tuple(chosen)] = 1
        
    return chosen_lst
    
    

In [15]:
test_set_good = generate_test_good(count_test_good)
test_set_nm = generate_nm_short(count_test_nm)
test_set_longone = generate_nm_long(count_test_longone)
test_set_nodep = generate_nodep_short(count_test_nodep)
test_set_long_nodep = generate_nodep_long(count_test_long_nodep)
test_set_long = generate_training_set(0,0,0,count_test_long, [])
test_set_atob = generate_unordered_short(count_test_atob)
training_set = generate_training_set(count_train_1, count_train_2, count_train_3, 0, test_set_good)
test_set_seen = choose_uniqe_subset(training_set, count_test_seen)


In [16]:
len(training_set)

48

In [17]:
test_set = []

for elt in test_set_good:
    test_set.append(["y", "good"] + elt)
    
for elt in test_set_nm:
    test_set.append(["n", "nm"] + elt)

for elt in test_set_longone:
    test_set.append(["n", "longone"] + elt)
    
for elt in test_set_nodep:
    test_set.append(["n", "nodep"] + elt)
    
for elt in test_set_long_nodep:
    test_set.append(["n", "long_nodep"] + elt)
    
for elt in test_set_long:
    test_set.append(["y", "long"] + elt)
    
for elt in test_set_atob:
    test_set.append(["n", "atob"] + elt)
    
for elt in test_set_seen:
    test_set.append(["y", "seen"] + elt)

test_set



[['y', 'good', 'di', 'bi', 'de', 'tu', 'po', 'to'],
 ['y', 'good', 'be', 'di', 'tu', 'po'],
 ['y', 'good', 'di', 'tu'],
 ['y', 'good', 'di', 'de', 'bi', 'po', 'to', 'tu'],
 ['y', 'good', 'gi', 'ko'],
 ['y', 'good', 'de', 'to'],
 ['y', 'good', 'gi', 'di', 'tu', 'ku'],
 ['y', 'good', 'be', 'bi', 'po', 'pu'],
 ['y', 'good', 'de', 'to'],
 ['y', 'good', 'gi', 'ko'],
 ['n', 'nm', 'bi', 'gi', 'ko'],
 ['n', 'nm', 'di', 'to', 'ko'],
 ['n', 'nm', 'de', 'gi', 'ku'],
 ['n', 'nm', 'di', 'to', 'po'],
 ['n', 'nm', 'gi', 'ku', 'to'],
 ['n', 'longone', 'bi', 'de', 'gi', 'ko', 'tu', 'po', 'to'],
 ['n', 'longone', 'gi', 'di', 'de', 'bi', 'pu', 'to', 'tu'],
 ['n', 'longone', 'di', 'gi', 'be', 'bi', 'po', 'pu', 'ko'],
 ['n', 'longone', 'di', 'gi', 'de', 'to', 'ko', 'tu', 'pu'],
 ['n', 'longone', 'gi', 'ge', 'de', 'di', 'tu', 'to', 'ko'],
 ['n', 'nodep', 'be', 'tu'],
 ['n', 'nodep', 'gi', 'de', 'gi', 'ku', 'to', 'po'],
 ['n', 'nodep', 'ge', 'gi', 'bi', 'pu', 'ku', 'tu'],
 ['n', 'nodep', 'bi', 'di', 'ge', 'k

In [18]:
def print_list_of_lists(lol):
    string = "[" + ", ".join("[" + ", ".join(['"' + y + '"' for y in x]) + "]" for x in lol) + "]"
    
    return string

In [23]:
print(print_list_of_lists(training_set))

[["di", "gi", "ku", "to"], ["gi", "ku"], ["bi", "po"], ["be", "bi", "pu", "po"], ["di", "gi", "ko", "to"], ["gi", "ge", "ku", "ko"], ["be", "pu"], ["de", "bi", "gi", "ku", "po", "to"], ["gi", "bi", "de", "to", "po", "ku"], ["bi", "pu"], ["ge", "bi", "gi", "ko", "po", "ku"], ["be", "gi", "ko", "po"], ["be", "po"], ["di", "de", "be", "po", "tu", "to"], ["be", "po"], ["gi", "ge", "ko", "ku"], ["de", "tu"], ["be", "po"], ["bi", "di", "to", "pu"], ["de", "di", "to", "tu"], ["be", "ge", "ku", "pu"], ["ge", "de", "gi", "ko", "tu", "ku"], ["di", "de", "tu", "to"], ["de", "di", "bi", "pu", "to", "tu"], ["ge", "bi", "gi", "ko", "pu", "ku"], ["ge", "de", "bi", "pu", "tu", "ku"], ["gi", "bi", "ge", "ko", "pu", "ku"], ["be", "gi", "ku", "pu"], ["ge", "gi", "be", "po", "ko", "ku"], ["di", "to"], ["gi", "be", "pu", "ko"], ["de", "bi", "gi", "ku", "po", "tu"], ["de", "tu"], ["bi", "pu"], ["gi", "ku"], ["gi", "bi", "po", "ko"], ["ge", "gi", "ko", "ku"], ["ge", "be", "gi", "ko", "pu", "ku"], ["be", "ge"

In [20]:
len(test_set)

50

In [21]:
# Number of test items that are
# the same as training items, for
# each length
#count_test_seen_1 = 10
#count_test_seen_2 = 10
#count_test_seen_3 = 10

# Number of test items that are like the training
# items but novel
#count_test_good_1 = 10
#count_test_good_2 = 10
#count_test_good_3 = 10

# Number of test items where the number of A's
# and B's differs: either 2 A's and 3 B's, or 
# vice versa
#count_test_nm_23 = 5
#count_test_nm_32 = 5

# Number of test items where the dependencies
# are wrong, specified by the length and the
# position where it is wrong.
# E.g., length3_wrong2 means that n=3 and that
# it is the 2nd position where the dependency is off
#count_test_nodep_length1_wrong1 = 5
#count_test_nodep_length2_wrong1 = 5
#count_test_nodep_length3_wrong1 = 5
#count_test_nodep_length3_wrong2 = 5

# Number of test items that are long but
# otherwise good
#count_test_long = 5

# Number of test items that are long and have an
# unmatching number of A's and B's
#count_test_longone_34 = 5
#count_test_longone_43 = 5

# Number of test items with n=4 and a dependency wrong
# in position 1, 2, or 3
#count_test_long_nodep_wrong1 = 5
#count_test_long_nodep_wrong2 = 5
#count_test_long_nodep_wrong3 = 5

# Number of test items where an A or B has been changed
#count_test_atob = 5
